In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Mirar la libreria sklearn.model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import linear_model

import warnings
#warnings.simplefilter('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Modelo de evolución de un disco protoplanetario en un sistema planetario
### A continuación se importan los datos de la evolución de un disco de gas y polvo que rodea una estrella central y la evolución de este en un sistema planetario.

### Preprocesamiento de los datos

In [3]:
high = pd.read_csv('datos/highp_finalresults.csv')
low = pd.read_csv('datos/lowp_finalresults.csv')

num_planetas_high = high.groupby(['ident']).size().get_values()
num_planetas_low = low.groupby(['ident']).size().get_values()

masa_plan_high = high.groupby(['ident'])['emepla(i)/emet'].sum().get_values()
masa_plan_low = low.groupby(['ident'])['emepla(i)/emet'].sum().get_values()

efic_masa_high = masa_plan_high/high.groupby(['ident'])['emed'].mean().get_values()
efic_masa_low = masa_plan_low/low.groupby(['ident'])['emed'].mean().get_values()

X_low = low.drop(['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie'],inplace=False,axis=1).groupby(['ident']).mean()
X_high = high.drop(['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie'],inplace=False,axis=1).groupby(['ident']).mean()

scaler = preprocessing.StandardScaler().fit(X_high)
scaler
Scale_high = scaler.transform(X_high)
scaler = preprocessing.StandardScaler().fit(X_low)
scaler
Scale_low = scaler.transform(X_low)

X_train, X_test, Y_train, Y_test = train_test_split(Scale_high, num_planetas_high, test_size=0.3)

## Modelo de estimación del número de planetas

###  Support Vector Machine con kernel gaussiano
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [4]:
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
gamma=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco=0
for c in C:
    for g in gamma:
        gauss = svm.SVC(C=c,kernel='rbf',gamma=g)
        gauss.fit(X_train,Y_train)
        Y_predict = gauss.predict(X_test)
        ii = Y_predict == Y_test
        if sum(ii) > sco:
            sco = sum(ii)
            mejor_g = g
            mejor_c = c
print('metodo: SVM rbf', '. hits:',sco, '. mejor gamma:',mejor_g, '. mejor C:', mejor_c)

metodo: SVM rbf . hits: 280 . mejor gamma: 0.01 . mejor c: 5


###  Support Vector Machine con kernel lineal
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [6]:
C = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
sco = 0
for c in C:
    linear = svm.SVC(C=c,kernel='linear')
    linear.fit(X_train,Y_train)
    Y_predict = linear.predict(X_test)
    ii = Y_predict == Y_test
    if sum(ii) > sco:
        sco = sum(ii)
        mejor_c = c
print('metodo: SVM linear', '. hits:',sco, '. mejor C:', mejor_c)

metodo: SVM linear . hits: 286 . mejor C: 0.1


###  Support Vector Machine con kernel polinomial
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [8]:
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
degr = [1, 2, 3, 4]
sco = 0
for c in C:
    for d in degr:
        poly = svm.SVC(C=c,kernel='poly',degree=d)
        poly.fit(X_train,Y_train)
        Y_predict = poly.predict(X_test)
        ii = Y_predict == Y_test
        if sum(ii) > sco:
            sco = sum(ii)
            mejor_d = d
            mejor_c = c
print('metodo: SVM poly', '. hits:',sco, '. mejor grado:',mejor_d, '. mejor C:', mejor_c)

metodo: SVM poly . hits: 281 . mejor grado: 1 . mejor C: 5


###  Lasso
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [7]:
alph = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco = 0
for a in alph:
    clf = linear_model.Lasso(alpha=a, tol=0.00001)
    clf.fit(X_train, Y_train)
    Y_predict = clf.predict(X_test)
    ii = 0
    for i in range(len(Y_predict)):
        ii+=round(Y_predict[i])==Y_test[i]
    if ii > sco:
        sco = ii
        mejor_a = a
print('metodo: Lasso', '. hits:',sco, '. mejor alpha:',mejor_a)

metodo: Lasso . hits: 234 . mejor alpha: 0.1


## Modelo de estimación de la masa total de los planetas

### Preprocesamiento de los datos

In [11]:
X_train_high, X_test_high, Y_train_high, Y_test_high = train_test_split(Scale_high, masa_plan_high, test_size=0.3)
X_train_low, X_test_low, Y_train_low, Y_test_low = train_test_split(Scale_low, masa_plan_low, test_size=0.3)

### Error

In [9]:
def error(Y_guess, Y_test):
    return (((Y_guess - Y_test)/Y_test)**2).mean()

### Support vector regression con score de ($\frac{1}{n} \sum^{n}_{i=1} ((y_i - \hat{y}_i)/y_i)^2)$)

In [13]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
Y_rbf = svr_rbf.fit(X_train_high, Y_train_high).predict(X_test_high)
err = error(Y_rbf, Y_test_high)
print ("Error de aproximación ", err)

Y_lin = svr_lin.fit(X_train_high, Y_train_high).predict(X_test_high)
err = error(Y_lin, Y_test_high)
print ("Error de aproximación ", err)

Y_poly = svr_poly.fit(X_train_high, Y_train_high).predict(X_test_high)
err = error(Y_poly, Y_test_high)
print ("Error de aproximación ", err)

Error de aproximación  21.7418155676
Error de aproximación  25.0506670874
Error de aproximación  7.6747212257


### Support Vector Regression con kernel gaussiano

In [14]:
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
gamma=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco=0
for c in C:
    for g in gamma:
        svr_rbf = SVR(kernel='rbf', C=c, gamma=g)
        Y_rbf = svr_rbf.fit(X_train_high, Y_train_high).predict(X_test_high)
        err = error(Y_rbf, Y_test_high)
        if (1/err) > sco:
            sco = 1/err
            mejor_g = g
            mejor_c = c
print('metodo: SVR rbf', '. score:',(1/sco), '. mejor gamma:',mejor_g, '. mejor c:', mejor_c)

metodo: SVR rbf . score: 1.19488727305 . mejor gamma: 0.1 . mejor c: 5


### Support Vector Regression con kernel lineal

In [15]:
C = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
sco = 0
for c in C:
    svr_lin = SVR(kernel='linear', C=c)
    Y_lin = svr_lin.fit(X_train_high, Y_train_high).predict(X_test_high)
    err = error(Y_lin, Y_test_high)
    if (1/err) > sco:
        sco = 1/err
        mejor_g = g
        mejor_c = c
print('metodo: SVR linear', '. score:',(1/sco), '. mejor c:', mejor_c)

metodo: SVR linear . score: 13.396375453 . mejor c: 0.01


### Support Vector Regression con kernel polinomial

In [16]:
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
degr = [1, 2, 3, 4]
sco = 0
for c in C:
    for d in degr:
        svr_poly = SVR(kernel='poly', C=c, degree=d)
        Y_poly = svr_poly.fit(X_train_high, Y_train_high).predict(X_test_high)
        err = error(Y_poly, Y_test_high)
        if (1/err) > sco:
            sco = 1/err
            mejor_g = g
            mejor_c = c
print('metodo: SVR poly', '. score:',(1/sco), '. mejor grado:',mejor_d, '. mejor c:', mejor_c)

metodo: SVR poly . score: 6.98360915293 . mejor grado: 1 . mejor c: 5
